GPU

In [2]:
import subprocess
import tensorflow as tf


def get_gpu_info():
    try:
        process = subprocess.Popen(['system_profiler', 'SPDisplaysDataType'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        return stdout.decode('utf-8')
    except Exception as e:
        return str(e)

print(get_gpu_info())

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print('GPU(s) used:')
    print(physical_devices)
else:
    print('No GPU used')


Graphics/Displays:

    Apple M2:

      Chipset Model: Apple M2
      Type: GPU
      Bus: Built-In
      Total Number of Cores: 10
      Vendor: Apple (0x106b)
      Metal Support: Metal 3
      Displays:
        Color LCD:
          Display Type: Built-in Liquid Retina Display
          Resolution: 2880 x 1864 Retina
          Main Display: Yes
          Mirror: Off
          Online: Yes
          Automatically Adjust Brightness: Yes
          Connection Type: Internal
        DELL E228WFP:
          Resolution: 1680 x 1050 (Widescreen Super eXtended Graphics Array Plus)
          UI Looks like: 1680 x 1050 @ 60.00Hz
          Mirror: Off
          Online: Yes
          Rotation: Supported


GPU(s) used:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Logger

In [ ]:
import sys
import atexit

class Logger(object):
    def __init__(self, filename='logfile.log'):
        self.terminal = sys.stdout
        self.log = open(filename, 'a')
        atexit.register(self.log.close)  # Register the file close method to be called when the program exits

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        self.terminal.flush()
        self.log.flush()

sys.stdout = Logger('logfile.log')

def example_function():
    print("This message will be logged to the file and printed on the console.")
    for i in range(5):
        print(f"Number: {i}")

if __name__ == '__main__':
    example_function()



In [7]:
!dir

 El volumen de la unidad T es DATOS
 El n�mero de serie del volumen es: 6648-9550

 Directorio de t:\Fede\SatNEx_code\tests

17/10/2023  10:17    <DIR>          .
17/10/2023  10:17    <DIR>          ..
09/10/2023  17:39             5.684 allLatencies_2gateways.csv
13/10/2023  03:59         1.806.350 allLatencies_3_gateways.csv
17/10/2023  10:19            15.309 becaria.ipynb
10/10/2023  14:41               124 logfile.log
               4 archivos      1.827.467 bytes
               2 dirs  168.024.330.240 bytes libres
